In [1]:
// object for variables so we don't run into problems when redclaring
const v = {
	// store for variables we obtained in asynchornous calls
	asynced: {},
};
// functions
const f = {};

undefined

In [2]:
v.const = {
	REPO_OWNER: 'RobotSail',
	REPO_NAME: 'apps-clone',
}

{ REPO_OWNER: 'RobotSail', REPO_NAME: 'apps-clone' }

In [3]:
var axios = require('axios');
v.headers = {
  Authorization:`bearer ${process.env.GITHUB_PAT}`
};
v.data = {
	query: `
		query {
			viewer {
				repositories(first: 100, privacy: PUBLIC, orderBy: {field: CREATED_AT, direction: DESC}) {
					nodes {
						name
						description
						url
					}
				}
			}
		}
	`
};
v.response = axios.get("https://api.github.com/graphql", {headers: v.headers, data: v.data}).then(resp => {
    console.log('received data', resp.data)
});



received data {
  data: {
    __schema: {
      queryType: [Object],
      mutationType: [Object],
      subscriptionType: null,
      types: [Array],
      directives: [Array]
    }
  }
}


undefined

In [4]:
var { gql } = require('@urql/core');
var { createClient } = require('@urql/core');
require('isomorphic-unfetch');

v.client = createClient({
    url: 'https://api.github.com/graphql',
    fetchOptions: () => {
        const gh_token = process.env.GITHUB_PAT;
        return {
            headers: { Authorization: gh_token ? `Bearer ${gh_token}` : '' }
        };
    },
});

Client {
  url: 'https://api.github.com/graphql',
  fetchOptions: [Function: fetchOptions],
  fetch: undefined,
  suspense: false,
  requestPolicy: 'cache-first',
  preferGetMethod: false,
  maskTypename: false,
  'operations$': [Function: source],
  reexecuteOperation: [Function: reexecuteOperation],
  createOperationContext: [Function: createOperationContext],
  createRequestOperation: [Function: createRequestOperation],
  executeRequestOperation: [Function: executeRequestOperation],
  executeQuery: [Function: executeQuery],
  executeSubscription: [Function: executeSubscription],
  executeMutation: [Function: executeMutation],
  query: [Function: query],
  readQuery: [Function: readQuery],
  subscription: [Function: subscription],
  mutation: [Function: mutation],
  subscribeToDebugTarget: [Function (anonymous)]
}

In [5]:
v.issuesQuery = gql`
# grab first 100 issues in operate first repo
query GetIssues ($owner: String!, $repo: String!, $limit: Int, $labels: [String!]) {
  repository(owner: $owner, name: $repo) {
    issues(first: $limit, labels: $labels) {
      totalCount
      pageInfo {	
        startCursor
        hasNextPage
        endCursor
      }
      edges {
        node {
          number
          body
          title
          author {
            login
          }
        }
      }
    }
  }
}
`;

{
  kind: 'Document',
  definitions: [
    {
      kind: 'OperationDefinition',
      operation: 'query',
      name: [Object],
      variableDefinitions: [Array],
      directives: [],
      selectionSet: [Object]
    }
  ],
  loc: {
    start: 0,
    end: 290,
    source: {
      body: '# GetIssues\n' +
        'query GetIssues($owner: String! $repo: String! $limit: Int $labels: [String!]) { repository(owner: $owner name: $repo) { issues(first: $limit labels: $labels) { totalCount pageInfo { startCursor hasNextPage endCursor } edges { node { number body title author { login } } } } } }',
      name: 'gql',
      locationOffset: [Object]
    }
  },
  __key: 3955687553
}

In [6]:
v.queryData = {};
v.client.query(v.issuesQuery)
    .toPromise()
    .then(r => v.queryData = r.data);


undefined

In [7]:
// get the first N issues
f.getIssues = async (gqlClient, limit = 1) => {
	// build query 
	const res = await gqlClient.query(v.issuesQuery, {
		owner: v.const.REPO_OWNER,
		repo: v.const.REPO_NAME,
		limit: limit
	})
		.toPromise()
		.then(r => {
			console.log(r);
			return r.data
		})
		.catch(e => console.log('error', e));
	await res
	v.asynced.issues = res.repository.issues.edges.map(({node}) => {
		return {
			number: node.number,
			body: node.body,
			title: node.title,
		}
	});
	return v.asynced.issues;
};

// needed to run async functions
$$.async();

{
	f.getIssues(v.client, 1);	
	console.log('issues', v.asynced.issues);
	$$done$$();
}


issues undefined


In [ ]:
console.log('variables at this point', v);

In [8]:


// return a list of files 
f.getFilesFromIssue = (issue) => {
	// extract a list of the files from the issue based on the following regex:
	// /`(@([a-zA-Z0-9_\-]+):(.+))`/g
	const fileRegex = /`(@([a-zA-Z0-9_\-]+):(.+))`/g;
	let files = [];
	// extract the string from group 3 of the regex
	let match;
	while (match = fileRegex.exec(issue)) {
		files.push(match[3]);
	}
	return files;
}


[Function (anonymous)]

In [9]:
// return a list of files
console.log(v.issues)
v.filesFromIssue = f.getFilesFromIssue(v.issues[0].body);
console.log(v.filesFromIssue);

undefined


TypeError: Cannot read properties of undefined (reading '0')

{
  operation: {
    key: 3976054725,
    query: { kind: 'Document', definitions: [Array], loc: [Object] },
    variables: { owner: 'RobotSail', repo: 'apps-clone', limit: 1 },
    kind: 'query',
    context: {
      url: 'https://api.github.com/graphql',
      fetchOptions: [Function: fetchOptions],
      fetch: undefined,
      preferGetMethod: false,
      suspense: false,
      requestPolicy: 'cache-first',
      meta: [Object]
    }
  },
  data: { repository: { issues: [Object], __typename: 'Repository' } },
  error: undefined,
  extensions: undefined,
  hasNext: false
}


In [ ]:
f.buildPrompt = (issue) => {
	/* The following is an example prompt that would be sent to OpenAI;
	==== BEGIN PROMPT ====
	# Listed below are:
	# 1. Explanation of how two files need to be changed
	# 2. The original files
	# 3. An updated version of the files with the described changes
	# 4. A '---' string, indicating the end of this document


	## Description of issues:
	@file1:/path/to/file1
	@file2:/path/to/file2

	The amount of CPU in @file1 needs to be increased to 512M
	The PVC storage amount in @file2 should be decreased to 10Gi

	## Original files:
	# /path/to/file1
	<contents of file1>

	# /path/to/file2
	<contents of file2>

	## Updated files:
	==== END PROMPT ====
	*/

	v.initialPrompt = `
# Listed below are:
# 1. Explanation of how two files need to be changed
# 2. The original files
# 3. An updated version of the files with the described changes
# 4. A '---' string, indicating the end of this document


## Description of issues:
${issue}`;
	// extract the files from the issue

};
